In [1]:
import json
from typing import List, Dict, Sized

import spacy
import jsonlines

In [2]:
def read_jsonl(file_path: str) -> List[Dict]:
    """
    Lit le fichier JSONL et retourne les données sous forme de liste de dictionnaires
    """
    data = []
    with jsonlines.open(file_path) as reader:
        for obj in reader:
            data.append(obj)
    return data

In [3]:
def is_valid_token(token: Sized) -> bool:
    """
    Vérifie si un token est valide
    """
    return (
            not token.is_stop  # pas de stopwords
            and token.is_alpha  # pas de caractères non-alphabétiques
            and token.pos_ != 'IN'  # pas de prépositions
    )

In [4]:
def post_precess_token(token: Sized) -> bool:
    """
    Enlève les résidus d'unités de mesure (mm, cm, m) présentes dans certains avis après avoir enlevé les nombres, et les tokens de moins de 3 caractères
    """
    return len(token) > 2 and token not in ['mm', 'cm', 'm']

In [5]:
def preprocess_text(text: str) -> List[str]:
    """
    Prétraite le texte : tokenisation, lemmatisation, suppression des stopwords
    """
    doc = nlp(text.lower())

    valid_tokens = [
        token.lemma_
        for token in doc
        if is_valid_token(token)
    ]

    processed_tokens = [
        token
        for token in valid_tokens
        if post_precess_token(token)
    ]

    return processed_tokens

In [6]:
def extract_relevant_fields(review: Dict) -> Dict:
    """
    Extrait uniquement les champs pertinents de chaque avis
    """
    return {
        "title": review.get("title", ""),
        "text": review.get("text", ""),
    }

In [7]:
def export_data(filepath: str, data: List[Dict]):
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [8]:
nlp = spacy.load("en_core_web_sm")

In [9]:
reviews = read_jsonl("data/reviews.jsonl")
processed_reviews_separated = []  # Pour séparer titre et texte
processed_reviews_combined = []  # Pour tous les tokens ensemble

In [10]:
for review in reviews:
    relevant_review = extract_relevant_fields(review)

    title_tokens = preprocess_text(relevant_review["title"])
    text_tokens = preprocess_text(relevant_review["text"])

    processed_review_separated = {
        "title_tokens": title_tokens,
        "text_tokens": text_tokens
    }
    processed_reviews_separated.append(processed_review_separated)

    all_tokens = title_tokens + text_tokens
    processed_reviews_combined.append(all_tokens)

In [11]:
export_data("post_process/processed_reviews.json", processed_reviews_separated)
export_data("post_process/processed_reviews_combined.json", processed_reviews_combined)